In [1]:
class Opion():
    
    def __init__(self):
        
        self.dataroot= r'/home/jara/DeepInPainting/paris_original_test' #image dataroot
        self.maskroot= r'/home/jara/DeepInPainting/mask/test_mask'
        self.batchSize= 1   # Need to be set to 1
        self.fineSize=256 # image size
        self.input_nc=3  # input channel size for first stage
        self.input_nc_g=6 # input channel size for second stage
        self.output_nc=3# output channel size
        self.ngf=64 # inner channel
        self.ndf=64# inner channel
        self.which_model_netD='basic' # patch discriminator
        
        self.which_model_netF='feature'# feature patch discriminator
        self.which_model_netG='unet_csa'# seconde stage network
        self.which_model_netP='unet_256'# first stage network
        self.triple_weight=1
        self.name='CSA_inpainting'
        self.n_layers_D='3' # network depth
        self.gpu_ids=[0]
        self.model='csa_net'
        self.checkpoints_dir=r'/home/jara/DeepInPainting/checkpoints' #
        self.norm='instance'
        self.fixed_mask=1
        self.use_dropout=False
        self.init_type='normal'
        self.mask_type='random'
        self.lambda_A=100
        self.threshold=5/16.0
        self.stride=1
        self.shift_sz=1 # size of feature patch
        self.mask_thred=1
        self.bottleneck=512
        self.gp_lambda=10.0
        self.ncritic=5
        self.constrain='MSE'
        self.strength=1
        self.init_gain=0.02
        self.cosis=1
        self.gan_type='lsgan'
        self.gan_weight=0.2
        self.overlap=4
        self.skip=0
        self.display_freq=1000
        self.print_freq=50
        self.save_latest_freq=5000
        self.save_epoch_freq=2
        self.continue_train=False
        self.epoch_count=1
        self.phase='train'
        self.which_epoch=46
        self.niter=20
        self.niter_decay=100
        self.beta1=0.5
        self.lr=0.0002
        self.lr_policy='lambda'
        self.lr_decay_iters=50
        self.isTrain=False

    

In [2]:
import time
from util.data_load import Data_load
from models.models import create_model
import torch
import os
import torchvision
from torch.utils import data
import torchvision.transforms as transforms
from IQA_pytorch import SSIM
import numpy as np
opt = Opion()
transform_mask = transforms.Compose(
    [transforms.Resize((opt.fineSize,opt.fineSize)),
     transforms.ToTensor(),
    ])
transform = transforms.Compose(
    [
     transforms.Resize((opt.fineSize,opt.fineSize)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)])
transform_ref = transforms.Compose(
    [transforms.RandomResizedCrop((opt.fineSize,opt.fineSize),scale=(0.8,1.0),ratio=(1,1)),
     transforms.ColorJitter(brightness=0.1,contrast=0.1,saturation=0.1,hue=0.1),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)])

dataset_test = Data_load(opt.dataroot, opt.maskroot,opt.dataroot, transform, transform_mask,transform_ref)
iterator_test = (data.DataLoader(dataset_test, batch_size=opt.batchSize,shuffle=True))
print(len(dataset_test))
model = create_model(opt)
total_steps = 0


2400
csa_net
initialize network with normal


/home/jara/DeepInPainting/models/networks.py:54: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, 0.0, gain)
/home/jara/DeepInPainting/models/networks.py:64: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias.data, 0.0)


initialize network with normal
Loading pre-trained network!
model [CSAModel] was created


In [3]:
load_epoch=46
model.load(load_epoch)

In [ ]:
save_dir = r'/home/jara/DeepInPainting/result'
if os.path.exists(save_dir) is False:
    os.makedirs(save_dir)

epoch=1
i=0
for image, mask,ref in (iterator_test):
    iter_start_time = time.time()
    image=image.cuda()
    mask=mask.cuda()
    mask=mask[0][0]
    mask=torch.unsqueeze(mask,0)
    mask=torch.unsqueeze(mask,1)
    mask=mask.bool()

    model.set_input(image,mask,ref) # it not only sets the input data with mask, but also sets the latent mask.
    model.set_ref_latent()
    model.set_gt_latent()
    model.test()
    
    real_A,ref_B,fake_B,fake_P,real_B=model.get_current_visuals()
    pic = (torch.cat([real_A, real_B,fake_P,fake_B], dim=0) + 1) / 2.0
    
    torchvision.utils.save_image(pic, '%s/Epoch_(%d)_(%dof%d).jpg' % (
    save_dir, epoch, total_steps + 1, len(dataset_test)), nrow=2)
    
    psnr=0
    mse=0
    if type(real_B)==torch.Tensor:
        mse=torch.mean((real_B-fake_B)**2)
    else:
        mse=np.mean((real_B-fake_B)**2)
    
    if mse==0:
        psnr=100
    else:
        psnr=10*torch.log10((2**2)/mse)
    errors=model.get_error()
    
    mod=SSIM(channels=3)
    score=mod(real_B,fake_B,as_loss=False)
    print('PSNR : %f, SSIM : %f' %
         (psnr,score))
    epoch=epoch+1


PSNR : 30.700304, SSIM : 0.924992
PSNR : 31.154312, SSIM : 0.902658
PSNR : 25.620840, SSIM : 0.736470
PSNR : 15.049090, SSIM : 0.499750
PSNR : 19.331890, SSIM : 0.522104
PSNR : 40.012634, SSIM : 0.971543
PSNR : 23.169899, SSIM : 0.649058
PSNR : 32.301964, SSIM : 0.944474
PSNR : 30.103264, SSIM : 0.943217
PSNR : 34.631504, SSIM : 0.929013
PSNR : 40.509445, SSIM : 0.980038
PSNR : 22.568932, SSIM : 0.827271
PSNR : 22.464277, SSIM : 0.687785
PSNR : 23.222698, SSIM : 0.709016
PSNR : 18.251179, SSIM : 0.534014
PSNR : 33.566242, SSIM : 0.910867
PSNR : 24.047813, SSIM : 0.768470
PSNR : 33.943954, SSIM : 0.967443
PSNR : 25.448044, SSIM : 0.846079
PSNR : 28.366989, SSIM : 0.795180
PSNR : 28.071087, SSIM : 0.949425
PSNR : 19.927322, SSIM : 0.578732
PSNR : 26.129539, SSIM : 0.860464
PSNR : 17.706833, SSIM : 0.621192
PSNR : 30.824871, SSIM : 0.919231
PSNR : 23.557041, SSIM : 0.832160
PSNR : 17.035950, SSIM : 0.502330
PSNR : 35.769558, SSIM : 0.951486
PSNR : 36.414532, SSIM : 0.969118
PSNR : 29.6784

PSNR : 29.694862, SSIM : 0.911452
PSNR : 26.947655, SSIM : 0.895137
PSNR : 25.708309, SSIM : 0.908634
PSNR : 21.141582, SSIM : 0.625874
PSNR : 21.702435, SSIM : 0.662820
PSNR : 38.131004, SSIM : 0.967707
PSNR : 19.409761, SSIM : 0.508309
PSNR : 14.110684, SSIM : 0.571516
PSNR : 37.717503, SSIM : 0.973505
PSNR : 26.005703, SSIM : 0.774643
PSNR : 28.387106, SSIM : 0.702793
PSNR : 19.912594, SSIM : 0.629471
PSNR : 31.667885, SSIM : 0.954386
PSNR : 24.732903, SSIM : 0.783469
PSNR : 23.340471, SSIM : 0.706328
PSNR : 38.915764, SSIM : 0.988910
PSNR : 29.308680, SSIM : 0.901587
PSNR : 31.055868, SSIM : 0.780806
PSNR : 27.075613, SSIM : 0.913912
PSNR : 31.890308, SSIM : 0.926431
PSNR : 27.604799, SSIM : 0.908213
PSNR : 27.366600, SSIM : 0.823939
PSNR : 18.138483, SSIM : 0.664963
PSNR : 28.024426, SSIM : 0.825961
PSNR : 25.385574, SSIM : 0.770357
PSNR : 32.107941, SSIM : 0.945993
PSNR : 26.112320, SSIM : 0.859092
PSNR : 28.280087, SSIM : 0.845529
PSNR : 27.383850, SSIM : 0.879530
PSNR : 30.3203

PSNR : 18.019764, SSIM : 0.657646
PSNR : 31.811769, SSIM : 0.884798
PSNR : 20.855474, SSIM : 0.560687
PSNR : 25.249292, SSIM : 0.752939
PSNR : 24.065418, SSIM : 0.560553
PSNR : 20.154799, SSIM : 0.644035
PSNR : 21.445913, SSIM : 0.638940
PSNR : 28.857973, SSIM : 0.909320
PSNR : 20.969532, SSIM : 0.563191
PSNR : 28.196095, SSIM : 0.910667
PSNR : 27.208061, SSIM : 0.786195
PSNR : 33.454308, SSIM : 0.905858
PSNR : 24.120848, SSIM : 0.758737
PSNR : 18.552408, SSIM : 0.651294
PSNR : 37.035118, SSIM : 0.978413
PSNR : 33.225185, SSIM : 0.953926
PSNR : 24.000977, SSIM : 0.700083
PSNR : 32.819340, SSIM : 0.959901
PSNR : 34.616226, SSIM : 0.953782
PSNR : 20.255951, SSIM : 0.741928
PSNR : 46.630188, SSIM : 0.988197
PSNR : 28.641119, SSIM : 0.939438
PSNR : 41.563686, SSIM : 0.987052
PSNR : 19.990810, SSIM : 0.688101
PSNR : 26.242432, SSIM : 0.815567
PSNR : 18.900017, SSIM : 0.585449
PSNR : 32.331383, SSIM : 0.893650
PSNR : 22.587843, SSIM : 0.744586
PSNR : 19.501087, SSIM : 0.693865
PSNR : 21.8666

PSNR : 21.322369, SSIM : 0.663561
PSNR : 18.855366, SSIM : 0.516549
PSNR : 28.841301, SSIM : 0.880885
PSNR : 22.601397, SSIM : 0.703446
PSNR : 22.965935, SSIM : 0.759663
PSNR : 18.443092, SSIM : 0.602482
PSNR : 27.968731, SSIM : 0.908417
PSNR : 19.297306, SSIM : 0.614844
PSNR : 20.971531, SSIM : 0.617986
PSNR : 43.316185, SSIM : 0.952185
PSNR : 22.014475, SSIM : 0.879219
PSNR : 19.094488, SSIM : 0.619420
PSNR : 17.590954, SSIM : 0.539278
PSNR : 22.368937, SSIM : 0.749475
PSNR : 31.788300, SSIM : 0.942924
PSNR : 17.130772, SSIM : 0.581600
PSNR : 19.170744, SSIM : 0.797625
PSNR : 29.624275, SSIM : 0.952298
PSNR : 22.134480, SSIM : 0.711134
PSNR : 25.646345, SSIM : 0.782273
PSNR : 22.304220, SSIM : 0.790733
PSNR : 19.459141, SSIM : 0.762435
PSNR : 20.968267, SSIM : 0.670462
PSNR : 21.824944, SSIM : 0.621153
PSNR : 39.392914, SSIM : 0.987155
PSNR : 23.731592, SSIM : 0.721293
PSNR : 26.390011, SSIM : 0.863269
PSNR : 18.265272, SSIM : 0.539663
PSNR : 27.986874, SSIM : 0.893867
PSNR : 36.7313

PSNR : 26.498259, SSIM : 0.764945
PSNR : 42.338432, SSIM : 0.986414
PSNR : 19.976170, SSIM : 0.505127
PSNR : 23.193893, SSIM : 0.627832
PSNR : 19.561115, SSIM : 0.676182
PSNR : 23.142822, SSIM : 0.742078
PSNR : 32.268730, SSIM : 0.906712
PSNR : 24.238270, SSIM : 0.681028
PSNR : 37.345718, SSIM : 0.932817
PSNR : 41.626286, SSIM : 0.970635
PSNR : 30.997580, SSIM : 0.920872
PSNR : 32.607521, SSIM : 0.901897
PSNR : 36.317646, SSIM : 0.979157
PSNR : 25.420687, SSIM : 0.910182
PSNR : 18.184097, SSIM : 0.725054
PSNR : 17.924540, SSIM : 0.530376
PSNR : 21.670359, SSIM : 0.726810
PSNR : 30.097584, SSIM : 0.905560
PSNR : 17.978754, SSIM : 0.544497
PSNR : 23.609638, SSIM : 0.730952
PSNR : 23.083658, SSIM : 0.742682
PSNR : 16.968296, SSIM : 0.656085
PSNR : 33.956371, SSIM : 0.951559
PSNR : 35.914513, SSIM : 0.945490
PSNR : 28.264011, SSIM : 0.888854
PSNR : 19.954647, SSIM : 0.678468
PSNR : 16.764566, SSIM : 0.535156
PSNR : 35.984436, SSIM : 0.962205
PSNR : 28.398312, SSIM : 0.794795
PSNR : 25.8896

PSNR : 26.318710, SSIM : 0.749085
PSNR : 26.539162, SSIM : 0.833746
PSNR : 27.339043, SSIM : 0.894099
PSNR : 22.240093, SSIM : 0.840764
PSNR : 20.571024, SSIM : 0.544231
PSNR : 27.995546, SSIM : 0.853300
PSNR : 43.697872, SSIM : 0.992205
PSNR : 25.394239, SSIM : 0.624724
PSNR : 28.965488, SSIM : 0.897143
PSNR : 25.948845, SSIM : 0.892571
PSNR : 21.732243, SSIM : 0.776127
PSNR : 25.286257, SSIM : 0.803369
PSNR : 22.489002, SSIM : 0.700346
PSNR : 15.770164, SSIM : 0.656097
PSNR : 23.462620, SSIM : 0.680304
PSNR : 33.390991, SSIM : 0.961189
PSNR : 28.420078, SSIM : 0.829265
PSNR : 30.987577, SSIM : 0.865338
PSNR : 29.728912, SSIM : 0.922987
PSNR : 22.396614, SSIM : 0.633307
PSNR : 22.150967, SSIM : 0.533318
PSNR : 32.795151, SSIM : 0.974528
PSNR : 19.508154, SSIM : 0.724590
PSNR : 18.878000, SSIM : 0.526984
PSNR : 34.829914, SSIM : 0.827024
PSNR : 32.325745, SSIM : 0.942604
PSNR : 24.626789, SSIM : 0.746261
PSNR : 20.550625, SSIM : 0.695052
PSNR : 24.506317, SSIM : 0.803060
PSNR : 26.7399